## Conversão de Dados

A conversão de tipos de dados é uma etapa fundamental no processamento e análise de dados em ambientes como o Databricks. Muitas vezes, os dados chegam em formatos que não são ideais para cálculos, comparações ou visualizações, tornando necessário transformar colunas para tipos mais adequados, como inteiros, decimais, datas ou booleanos. Compreender como realizar essas conversões de forma eficiente garante maior precisão nas análises, evita erros e facilita a integração entre diferentes fontes de dados. Nesta seção, abordaremos as principais técnicas e boas práticas para conversão de dados utilizando PySpark, destacando exemplos práticos e explicando as diferenças entre os tipos mais comuns.

### Renomeando Colunas withColumnRenamed e Alias

Renomear colunas em DataFrames é uma tarefa comum ao trabalhar com dados no Databricks. Utilizando métodos como `withColumnRenamed` e `alias`, é possível tornar os nomes das colunas mais descritivos e adequados para análise, facilitando a leitura e o entendimento dos dados ao longo do processo de transformação.

Exemplo:

```py
df.withColumnRenamed("coluna_antiga", "coluna_nova")
```

In [0]:
path = "/Volumes/workspace/default/tutorial/anac"
df = spark.read.json(f"{path}/V_OCORRENCIA_AMPLA.json")
display(df)

In [0]:
print(df.columns)

In [0]:
renomeado = df.withColumnRenamed('Aerodromo_de_Destino', 'Destino') \
    .withColumnRenamed('Aerodromo_de_Origem', 'Origem') \
    .withColumnRenamed('Danos_a_Aeronave', 'Danos')\
    .withColumnRenamed('UF', 'Estado')

display(renomeado)

In [0]:
print(renomeado.columns)

In [0]:
print(df.columns)

In [0]:
# Selecionando apenas algumas colunas e ja renomeando com .alias

Teste = df.select(
        df.Aerodromo_de_Destino.alias('Destino'),
        df.Nome_do_Fabricante,
        df.Aerodromo_de_Origem.alias('Origem'),
        df.UF.alias('Estado'),
        df.Numero_da_Ocorrencia,
        df.Matricula.alias("Registro")
)

display(Teste)

### Conversão de Dados método cast + withColumn

A conversão de tipos de dados é uma etapa fundamental no processamento de dados com Spark DataFrames. Muitas vezes, é necessário alterar o tipo de uma coluna para garantir a correta análise e manipulação dos dados, como converter uma coluna de string para inteiro ou data. No Databricks, isso pode ser feito utilizando o método `cast` em conjunto com `withColumn`, permitindo transformar o tipo de uma coluna de forma eficiente e flexível durante o pipeline de transformação.

A função cast no PySpark aceita vários tipos de dados para a conversão:

* **nullable**: isso indica que alguns dos registros nessa coluna podem ter valores nulos.
* **string ou varchar**: Texto.
* **integer ou int**: Numero inteiro.
* **long**: Numero inteiro longo.
* **double**: precisão dupla.
* **float**: precisão simples.
* **decimal**: decimal.
* **timestamp**: Data e hora.
* **date**: Somente a Data.
* **boolean**: booleana (True e False) 0=False e 1=True.

Qual diferença entre esses que trazem numeros decimais no geral

**float** (Precisão Simples 32 bits na memória):
* Mínimo: 1.40129846432 x 10^-45
* Máximo: 3.40282346639 x 10^38

**double** (Precisão Dupla 64 bits na memória):
* Mínimo: 4.94065645841 x 10^-324
* Máximo: 1.79769313486 x 10^308

**decimal**: pode escolher quantas casas decimais aparece no resultado.

O decimal usa uma representação decimal, o que permite maior precisão e evita muitos dos problemas de arredondamento associados ao float.

In [0]:
path = "/Volumes/workspace/default/tutorial/bike-store"

In [0]:
# Carga DF sem .option("inferSchema", "true") sendo assim virá tudo como string.
orders_df = spark.read. \
    format("csv") \
    .option("header", "true") \
    .load(f"{path}/orders.csv")

display(orders_df)

In [0]:
orders_df.printSchema()

In [0]:
# Converter Dados com withColumn
orders_df = orders_df.withColumn('order_id', orders_df.order_id.cast('int') ) 
orders_df = orders_df.withColumn('customer_id', orders_df.customer_id.cast('integer') ) 
orders_df = orders_df.withColumn('order_date', orders_df.order_date.cast('date'))
orders_df = orders_df.withColumn('required_date', orders_df.required_date.cast('date'))
display(orders_df)

In [0]:
orders_df.printSchema()

In [0]:
#Converter Dados com .Select  
orders_df2 = spark.read.\
    format("csv")\
    .option("header", "true")\
    .load(f"{path}/orders.csv")

display(orders_df2)

In [0]:
# .select (Converte e ja seleciona as colunas, pode renomear com Alias tambem)
Conversao = orders_df2.select(
    orders_df2.order_id.cast("integer"),
    orders_df2.order_date.cast("date"),
    orders_df2.store_id.cast("int"),
    orders_df2.staff_id.cast("int").alias("IdLoja"),
    orders_df2.order_status.cast("int").alias("Status_Entrega"),
)

display(Conversao)

###Conversões númericas Avançadas

A manipulação de dados numéricos em Spark DataFrames frequentemente exige conversões avançadas entre diferentes tipos numéricos, como float, double e decimal. Essas conversões são essenciais para garantir precisão em cálculos, evitar problemas de arredondamento e adequar os dados ao contexto analítico desejado. Nesta seção, exploraremos técnicas e boas práticas para realizar conversões numéricas avançadas utilizando PySpark, destacando as diferenças entre os principais tipos e exemplos práticos de uso.

In [0]:

Items = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/order_items.csv")

display(Items)

In [0]:
print(Items.columns)
Items.printSchema()

In [0]:
# Formato double
fracionado = Items.withColumn('Fracao', Items.list_price / 2.03)
display(fracionado)

In [0]:
# Criando novas colunas ja convertidas
# O valor decimal terá um máximo de 10 dígitos no total (incluindo os dígitos à esquerda e à direita do ponto decimal) e 2 dígitos à direita do ponto decimal.
fracionado2 = fracionado\
    .withColumn('Double' , fracionado.Fracao) \
    .withColumn('Int'    , fracionado.Fracao.cast('integer') ) \
    .withColumn('Float'  , fracionado.Fracao.cast('float') ) \
    .withColumn('Decimal', fracionado.Fracao.cast('decimal(10,2)'))

display(fracionado2)

In [0]:
# Substituindo o double por decimal no Df
SubDouble =  fracionado.withColumn('Fracao', fracionado.Fracao.cast('decimal(10,2)'))
display(SubDouble)

###Arredondamento de Numeros

O arredondamento de números é uma operação fundamental em análises de dados, especialmente quando lidamos com valores decimais em grandes volumes de informações. No contexto do PySpark e Databricks, existem funções específicas que permitem controlar a precisão dos números, facilitando a apresentação dos resultados e evitando problemas de interpretação causados por casas decimais excessivas. Nesta seção, veremos como aplicar diferentes métodos de arredondamento em DataFrames, destacando suas aplicações práticas e diferenças principais.

* **floor**: (arredondamento para baixo)
* **ceil**:  (arredondamento para cima)
* **round**: (arredondamento Automatico)

In [0]:
display(fracionado2)

In [0]:
# Aproveitar script fracionado2 para Treinar o floor, ceil e round 

from pyspark.sql.functions import floor, ceil, round
Arred = fracionado2 \
    .withColumn('ArredAcima', ceil(fracionado2.Double)) \
    .withColumn('ArredAbaixo', floor(fracionado2.Double)) \
    .withColumn('ArredAutomatico', round(fracionado2.Double))

display(Arred)

##Trabalhando com Datas

https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

https://spark.apache.org/docs/latest/sql-ref-datatypes.html

O tratamento de datas é uma etapa essencial no processamento de dados, pois informações temporais estão presentes em praticamente todos os domínios analíticos. No Databricks, trabalhar com datas e timestamps em DataFrames Spark permite realizar operações como conversão de formatos, extração de partes da data (ano, mês, dia), cálculos de diferença entre datas, filtragem por períodos e muito mais. Compreender como manipular corretamente esses tipos de dados é fundamental para garantir análises precisas, enriquecer relatórios e facilitar a tomada de decisões baseada em tempo.

Referencias:

https://docs.databricks.com/en/sql/language-manual/functions/from_utc_timestamp.html

https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.functions.from_utc_timestamp.html

In [0]:
orders_df = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/orders.csv")

display(orders_df)

In [0]:
orders_df.printSchema()

In [0]:
# Extraindo uma fração de uma data em novas Colunas 
from pyspark.sql.functions import year, month, dayofmonth

orders_df = orders_df \
    .withColumn("ano_pedido", year("order_date")) \
    .withColumn("mes_pedido", month("order_date")) \
    .withColumn("dia_pedido", dayofmonth("order_date"))

display(orders_df)

In [0]:
# Usando a classe datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from datetime import date

@udf(IntegerType())
def get_year(date: date):
    return date.year

@udf(IntegerType())
def get_month(date: date):
    return date.month

@udf(IntegerType())
def get_day(date: date):
    return date.day

orders_df2 = orders_df \
    .withColumn("ano_pedido", get_year("order_date")) \
    .withColumn("mes_pedido", get_month("order_date")) \
    .withColumn("dia_pedido", get_day("order_date"))

display(orders_df2)

In [0]:
# Simulando mater registro de quando os dados foram carregados (Muito comum no dia a dia para rastreio se os dados foram atualizados ou nao)
# data_hora_br: Formato brasil -3h

from pyspark.sql.functions import current_date, current_timestamp, expr

orders_df = orders_df \
    .withColumn("data_atual", current_date()) \
    .withColumn("data_hora_atual", current_timestamp()) \
    .withColumn("data_hora_br", expr("from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')"))

display(orders_df)

In [0]:
# Extraindo horas minutos e Segundos 
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second

orders_df = orders_df \
    .withColumn("hora_carga", hour("data_hora_br")) \
    .withColumn("minuto_carga", minute("data_hora_br")) \
    .withColumn("segundo_carga", second("data_hora_br"))

display(orders_df)


### Comandos e Formatos de Datas avançado

https://docs.databricks.com/en/sql/language-manual/functions/cast.html

https://spark.apache.org/docs/latest/sql-ref-datetime-pattern.html

O tratamento avançado de datas é fundamental para análises temporais precisas em ambientes de Big Data como o Databricks. Compreender os diferentes formatos, comandos e funções disponíveis para manipulação de datas permite realizar operações como conversão de tipos, extração de partes específicas (ano, mês, dia, hora), cálculos de intervalos e ajustes de timezone. Nesta seção, apresentamos as principais técnicas e boas práticas para trabalhar com datas em PySpark, incluindo exemplos práticos e referências úteis para consulta.

In [0]:
orders_df = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/orders.csv")

display(orders_df)

In [0]:
# Formato PT-Br

from pyspark.sql.functions import date_format

orders_df = orders_df.withColumn(
    "data_formatada",
    date_format(orders_df.order_date, "dd/MM/yyyy")
)

display(orders_df)


In [0]:
# Exemplos outros formatos
# MMM > mes abrev 
# "Jan Feb Mar Apr May Jun Jul Aug Sep Oct Nov Dec",
# "Jan Fev Mar Abr Mai Jun Jul Ago Set Out Nov Dez"

# MMMM > mes completo
# "January February March April May June July August September October November December",
# "Janeiro Fevereiro Março Abril Maio Junho Julho Agosto Setembro Outubro Novembro Dezembro"


# E > dia da semana 
# "Sun Mon Tue Wed Thu Fri Sat", 
# "Dom Seg Ter Qua Qui Sex Sáb"

# EEEE > dia da semana completo
# "Sunday Monday Tuesday Wednesday Thursday Friday Saturday", 
# "Domingo Segunda-feira Terça-feira Quarta-feira Quinta-feira Sexta-feira Sábado"

from pyspark.sql.functions import date_format 

orders_df = orders_df \
        .withColumn("Exemplo1", date_format(orders_df.order_date,"yyyy MM dd")) \
        .withColumn("Exemplo2", date_format(orders_df.order_date,"MM/dd/yyyy HH:mm")) \
        .withColumn("Exemplo3", date_format(orders_df.order_date,"MM-yyyy")) \
        .withColumn("Exemplo4", date_format(orders_df.order_date,"yyyy MMM dd")) \
        .withColumn("Exemplo5", date_format(orders_df.order_date,"yyyy MMMM dd E")) \
        .withColumn("Exemplo6", date_format(orders_df.order_date,"MMM")) \
        .withColumn("Exemplo7", date_format(orders_df.order_date,"MMM-yyyy")
)   

display(orders_df)


### Traduçoes para portugues

A tradução de termos técnicos e comandos para o português é uma prática importante para facilitar o entendimento e a aprendizagem de conceitos relacionados ao processamento de dados, especialmente para quem está iniciando ou prefere estudar na língua nativa. Nesta seção, apresentaremos traduções dos principais termos, funções e comandos utilizados em PySpark e Databricks, tornando o conteúdo mais acessível e promovendo uma melhor compreensão dos processos envolvidos na manipulação e análise de dados.

In [0]:
# Conversao para portugues método when 
from pyspark.sql.functions import when
orders_df2 = orders_df.withColumn(
    "Exemplo6",
    when(orders_df.Exemplo6 == "Jan","Jan")
    .when(orders_df.Exemplo6 == "Feb","Fev")
    .when(orders_df.Exemplo6 == "Mar","Mar")
    .when(orders_df.Exemplo6 == "Apr","Abr")
    .when(orders_df.Exemplo6 == "May","Mai")
    .when(orders_df.Exemplo6 == "Jun","Jun")
    .when(orders_df.Exemplo6 == "Jul","Jul")
    .when(orders_df.Exemplo6 == "Aug","Ago")
    .when(orders_df.Exemplo6 == "Sep","Set")
    .when(orders_df.Exemplo6 == "Oct","Out")
    .when(orders_df.Exemplo6 == "Nov","Nov")
    .when(orders_df.Exemplo6 == "Dec","Dez")
    .otherwise(orders_df.Exemplo6)
)

display(orders_df2)


### Diferença entre Datas Adição e Subtração

A manipulação de datas é uma tarefa recorrente em análises de dados, especialmente quando precisamos calcular intervalos, somar ou subtrair períodos de tempo. No Databricks, é possível realizar operações de adição e subtração de datas utilizando operadores e funções específicas, como `+`, `-`, `date_add`, `date_sub` e intervalos. Compreender como aplicar essas operações de forma correta é fundamental para extrair insights temporais, calcular idades, prazos, identificar tendências e realizar comparações entre eventos ao longo do tempo. Nesta seção, veremos exemplos práticos de como adicionar e subtrair datas em DataFrames e consultas SQL no Databricks.

In [0]:
orders_df = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/orders.csv")

display(orders_df)

In [0]:
from pyspark.sql.functions import datediff, months_between, year

orders_df = orders_df \
    .withColumn(
        "diferenca_dias",
        datediff("required_date", "order_date")
    ) \
    .withColumn(
        "diferenca_meses",
        months_between("required_date", "order_date")
    ) \
    .withColumn(
        "diferenca_anos",
        year("required_date") - year("order_date")
    )

display(orders_df)

In [0]:
# Simulando diferença de alguma coluna de data com a data Atual 
from pyspark.sql.functions import datediff, months_between, year ,current_date

orders_df = orders_df \
    .withColumn("diferenca_dias", datediff(current_date(), "required_date")) \
    .withColumn("diferenca_meses", months_between(current_date(), "required_date")) \
    .withColumn("diferenca_anos", year(current_date()) -  year("required_date"))

display(orders_df)

### Adicionar intervalos em Datas

https://docs.databricks.com/en/sql/language-manual/functions/cast.html

Adicionar intervalos de tempo a datas é uma operação comum em análises temporais, permitindo calcular prazos, prever eventos futuros ou ajustar períodos de análise. No Databricks, é possível somar ou subtrair intervalos de dias, meses, anos e outros componentes diretamente em colunas de DataFrames ou em consultas SQL, utilizando funções específicas para manipulação de datas. Compreender como aplicar corretamente esses intervalos é fundamental para garantir precisão em cálculos temporais e enriquecer as análises de dados.


In [0]:
orders_df = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/orders.csv")

display(orders_df)

In [0]:
from pyspark.sql.functions import expr

# Adicionar coluna 'previsao_entrega' com 5 dias adicionados à 'order_date'
# Adicionar coluna 'pagamento' com 1 mês adicionado à 'order_date'
# Adicionar coluna 'devolucao' com 1 Ano adicionado à 'order_date'

orders_df = orders_df \
    .withColumn("previsao_entrega", expr("order_date + interval 5 days")) \
    .withColumn("pagamento", expr("order_date + interval 1 month")) \
    .withColumn("max_devolucao", expr("order_date + interval 1 year"))

display(orders_df)

In [0]:
# Simulação rápida tirando intervalos
orders_df = orders_df \
    .withColumn("previsao_entrega", expr("order_date - interval 5 days")) \
    .withColumn("pagamento", expr("order_date - interval 1 month")) \
    .withColumn("max_devolucao", expr("order_date - interval 1 year"))

display(orders_df)


### Datas em SQL

O tratamento de datas em SQL é essencial para análises temporais, permitindo filtrar, agrupar, calcular intervalos e manipular informações baseadas em tempo diretamente nas consultas. No Databricks, é possível utilizar funções SQL específicas para converter formatos, extrair partes da data, realizar operações de adição e subtração de períodos, além de comparar e transformar valores temporais de forma eficiente. Nesta seção, apresentamos as principais técnicas e comandos para trabalhar com datas em SQL, facilitando a construção de análises robustas e precisas.

In [0]:
orders_df = spark.read. \
    format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(f"{path}/orders.csv")

display(orders_df)

In [0]:
orders_df.createOrReplaceTempView("tabela_ordens")

In [0]:
%sql 
select * from tabela_ordens

In [0]:
%sql 
-- extraindo partes de uma data 
SELECT *,
      YEAR(order_date) AS Ano,
      MONTH(order_date) AS Mes,
      DAY(order_date) AS Dia
FROM tabela_ordens


In [0]:
%sql 
-- para treinar   
-- traduções: now ou  current_timestamp  = carimbo de data/hora atual
-- muito usado para ter coluna ou colunas de quando os dados foram carregados para base, arquivos banco de dados etc
--  converter fuso horário from_utc_timestamp

select * ,
  now() as agora,
  current_timestamp() AS agora2,
  from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo') AS agora_PTBR,
  date_format(
    from_utc_timestamp(
      current_timestamp(),
      'America/Sao_Paulo'
    ),
    'yyyy-MM-dd HH:mm:ss'
    ) AS agora_formatado,
  date_format(
    from_utc_timestamp(
      current_timestamp(),
      'America/Sao_Paulo'
    ),
    'dd/MM/yyyy'
  ) AS agora_formatado_brasil,
  date_format(
    from_utc_timestamp(
      current_timestamp(),
      'America/Sao_Paulo'
    ),
    'dd/MM/yyyy HH:mm:ss'
  ) AS agora_formatado_brasil_Completo,
  year(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as ano,
  month(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as mes,
  day(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as dia,
  hour(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as hora,
  minute(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as minuto,
  second(from_utc_timestamp(current_timestamp(), 'America/Sao_Paulo')) as segundo
from tabela_ordens

In [0]:
%sql 
-- dados maior que certa data 
-- Obs o padrao é ano-mes-dia
SELECT * FROM tabela_ordens WHERE order_date >= '2016-01-01'


In [0]:
%sql 
-- Quantos anos tenho de registro na base?
SELECT distinct 
    year(order_date) 
FROM tabela_ordens

In [0]:
%sql 
-- Dados entre 2017  e 2018
SELECT * FROM tabela_ordens
WHERE year(order_date) BETWEEN 2017 and 2018

In [0]:
%sql 
-- Dados entre datas espesificas 
SELECT * FROM tabela_ordens
WHERE order_date BETWEEN '2017-01-04' and '2018-01-01' 
 -- Obs o padrao é ano-mes-dia

In [0]:
%sql
select * ,
    date_format(order_date, 'yyyy-MM-dd HH:mm:ss') AS ptbr,
    date_format(order_date, 'dd/MM/yyyy') AS dataformatada,
    date_format(order_date, 'dd/MM/yyyy HH:mm:ss') AS datahora
from tabela_ordens

In [0]:
# Salvar consulta em um DF para trabalhar com Python ou salvar como qualqer tipo de arquivo 

resultado = spark.sql("""
select * ,
    date_format(order_date, 'yyyy-MM-dd HH:mm:ss') AS ptbr,
    date_format(order_date, 'dd/MM/yyyy') AS dataformatada,
    date_format(order_date, 'dd/MM/yyyy HH:mm:ss') AS datahora
from tabela_ordens
""")

display(resultado)